In [104]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore

from langchain.document_loaders import PyPDFDirectoryLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from langchain_cohere import CohereEmbeddings
import os

In [94]:
os.environ["COHERE_API_KEY"]="AVfEN48Hcv0D0BMyG9uY8WFo1sVSppR7UCTQzdGz"
os.environ["PINECONE_API_KEY"]="a1180ea3-b608-4431-976e-a780f812d5d2"
os.environ["PINECONE_INDEX_NAME"] = "testing"
os.environ["PINECONE_API_ENV"]= "gcp-starter"


In [74]:
def load_pdf(data):
    loader = PyPDFDirectoryLoader(data
                    
                    )
    
    documents = loader.load()

    return documents

In [88]:
extracted_data = load_pdf(r"C:\Users\User pc\Desktop\streamlit_projects\End_to_end_Medial_Chatbot_LLama2\data")

In [89]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [90]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 7020


In [98]:
def cohere_embeddings():
    embeddings = CohereEmbeddings()

    return embeddings

In [99]:
embeddings = cohere_embeddings()

In [101]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 4096


In [106]:
index_name = "medicalchatbot"

In [107]:
docsearch = PineconeVectorStore.from_documents(text_chunks, embeddings, index_name=index_name)

In [108]:
query = "What are Allergies"
docs=docsearch.similarity_search(query)

In [109]:
print("Result", docs)

Result [Document(page_content='Allergy and Immunology. Boston: Little, Brown and Co.,\n1995.\nNovick, N. L. You Can Do Something About Your Allergies.\nNew York: Macmillan, 1994.\nWeil, A. Natural Health, Natural Medicine: A Comprehensive\nManual for Wellness and Self-Care. New York: Houghton\nMifflin, 1995.\nRichard Robinson\nAllergies\nDefinition\nAllergies are abnormal reactions of the immune sys-\ntem that occur in response to otherwise harmless sub-stances.', metadata={'page': 127.0, 'source': 'C:\\Users\\User pc\\Desktop\\streamlit_projects\\End_to_end_Medial_Chatbot_LLama2\\data\\Gale Encyclopedia of Medicine. Vol. 1. 2nd Edition ( PDFDrive ).pdf'}), Document(page_content='KEY TERMS\nAllergen —A substance that causes an allergy.\nAnaphylaxis —A sudden, life-threatening allergic\nreaction.\nHallucination —A false or distorted perception of\nobjects, sounds, or events that seems real. Halluci-nations usually result from drugs or mental disor-ders.\nHistamine —A chemical released f

In [110]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [111]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}